In [55]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, iqr
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [56]:
df = pd.read_csv("RecGym.csv")
df.head()

,Subject,Position,Session,A_x,A_y,A_z,G_x,G_y,G_z,C_1,Workout
0,1,wrist,1,0.501125,0.500625,0.500500,0.500156,0.499962,0.500313,0.501227,Null
1,1,wrist,1,0.500250,0.499750,0.501250,0.499531,0.500550,0.500081,0.501584,Null
2,1,wrist,1,0.500250,0.499375,0.501875,0.499613,0.500231,0.500119,0.501508,Null
3,1,wrist,1,0.499625,0.499750,0.500875,0.499494,0.500350,0.500588,0.502032,Null
4,1,wrist,1,0.500000,0.499000,0.501000,0.499650,0.500588,0.499769,0.501756,Null


In [57]:
df = df[df['Workout'] != "Null"].reset_index(drop=True)
df['Workout'].unique()

array(['LegCurl', 'Squat', 'Adductor', 'LegPress', 'ArmCurl',
       'BenchPress', 'RopeSkipping', 'Walking', 'Running', 'StairClimber',
       'Riding'], dtype=object)

In [58]:
sensors = ['A_x','A_y','A_z','G_x','G_y','G_z','C_1']

In [59]:
def extract_features(segment):
    feats = {}
    for col in sensors:
        data = segment[col].values
        feats[col + "_mean"]     = data.mean()
        feats[col + "_std"]      = data.std()
        feats[col + "_min"]      = data.min()
        feats[col + "_max"]      = data.max()
        feats[col + "_median"]   = np.median(data)
        feats[col + "_var"]      = data.var()
        feats[col + "_skew"]     = skew(data)
        feats[col + "_kurtosis"] = kurtosis(data)
        feats[col + "_rms"]      = np.sqrt(np.mean(data**2))
        feats[col + "_iqr"]      = iqr(data)
    return feats


In [ ]:
rows = []

for (sub, pos, sess), group in df.groupby(['Subject','Position','Session']):
    
    group = group.reset_index(drop=True)
    prev = "Null"
    seg_id = 0

    for i in range(len(group)):
        act = group.loc[i,'Workout']

        # Skip all Null rows entirely
        if act == "Null":
            prev = "Null"
            continue

        # Start a new segment when activity changes (and not Null)
        if act != prev:
            seg_id += 1
            start = i

        prev = act

        # end of segment?
        last = (i == len(group)-1)
        next_change = (not last) and (group.loc[i+1,'Workout'] != act)

        if last or next_change:
            segment = group.iloc[start:i+1]
            feats = extract_features(segment)

            feats["Subject"] = sub
            feats["Position"] = pos
            feats["Session"] = sess
            feats["InstanceID"] = seg_id
            feats["Workout"] = act

            rows.append(feats)

features_df = pd.DataFrame(rows)
print(features_df.shape)

In [ ]:
features_df = features_df.dropna()

In [ ]:
features_df.head()

In [ ]:
# y = labels (Workout name)
y = features_df["Workout"]

# X = all numeric feature columns (drop Subject, Position, Session, InstanceID, Workout)
non_features = ["Subject", "Position", "Session", "InstanceID", "Workout"]
X = features_df.drop(columns=non_features)


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA()
pca.fit(X_scaled)

plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("Number of components")
plt.ylabel("Cumulative explained variance")
plt.grid(True)
plt.show()

In [ ]:
X_pca = pca.transform(X_scaled)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)
pred_rf = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, pred_rf))
print(classification_report(y_test, pred_rf))

In [ ]:
svm = SVC(kernel='rbf')
svm.fit(X_train, y_train)
pred_svm = svm.predict(X_test)

print("SVM Accuracy:", accuracy_score(y_test, pred_svm))
print(classification_report(y_test, pred_svm))


In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
pred_knn = knn.predict(X_test)

print("KNN Accuracy:", accuracy_score(y_test, pred_knn))
print(classification_report(y_test, pred_knn))

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(y_test, pred_rf), annot=True, cmap="Blues", fmt="d")
plt.title("Random Forest - Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()